# Numpy Tutorial
Joshua Stough, 2021

[Numpy](https://numpy.org/) is a powerful toolkit for the handling of large N-dimensional arrays or matrices, conveniently wrapped in Python. Given the relative inefficiencies of large Pythonic lists of numbers, Numpy [`ndarray`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html)s are the backbone of scientific computation in Python. In this notebook we'll first show some time comparisons between `ndarray` and Python list basic functions, then explore a some of the very useful Numpy methods. Implicit in some of the later material will be the use of Matplotlib to visualize some of the multidimensional array data that we're manipulating.

1. [**Speed Comparisons**](#speedup)
1. [**Numpy Essentials**](#essentials)

## Imports

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import sys
# Alternative to using the timeit magic command:
# from timeit import timeit

<a id='speedup'></a>
## Speed Comparisons
We know Python lists as a really easy-to-use list implementation including powerful object-oriented and functional methods, where elements of a list can be of any type. This flexibility of the list type is accomplished through the use of pointers in a way that is hidden from the programmer--under the hood. Numpy on the other hand constrains all elements of an `ndarray` to be of the same type; under the hood, numpy arrays can be stored in contiguous memory with constant access time. 

A tremendous speedup results from executing numpy's compiled library code on contiguous memory, versus Python's dynamically interpreted bytecode implementations. 

To demonstrate this, we'll make a Python list of random integers using numpy's [randint](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html), a (uniformly distributed) random number generator. We'll also make a numpy `ndarray` copy of that list. This copy will be in memory that is independent of the list, which we are just copying the values from. 

Then we can test certain functionality on the `list` and `ndarray` collections separately, using IPython's [`timeit` magic command](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit).. 

In [ ]:
# Make a list of random integers in [0,10M)
lyst = [np.random.randint(0,10000000) for x in range(1000)]
np_lyst = np.array(lyst)

In [ ]:
%%timeit
min(lyst)

The expression we're timing is `min(lyst)`, which is the Python function to return the minimum of any iterable in Python. The default output of `timeit` tells us with some degree of certainty how much wall time calculating `min(lyst)` is likely to take, probably in units of microseconds $\mu s$. `timeit` computes this by taking several runs (default 7) and each time executing the expression potentially thousands of times to get to some accuracy in the measurement.

In [ ]:
%%timeit
np.min(np_lyst)

You should notice a significant speedup in the `np.min` result over the Python `min` result. On my machine I observe ~3x speedup on a list of length 1000. In that there is always some baseline overhead cost due to the Python interpreter (as opposed to compiled library calls), you may notice that the speedup improves the larger the list size is made to be. I observed a ~12x speedup on a list size of 10000, and a 15-20x speedup beyond 100000. 

In [ ]:
np_lyst.dtype

In [ ]:
# Using getsizeof to ask Python the size in bytes of a Python variable.
sys.getsizeof(lyst[0])

In [ ]:
type(lyst[0])

One significant note here is that there is a big precision reduction: the *integer* primitive in Python is for arbitrary magnitude integral values and requires 28 bytes, while the implicitly determined `np.int64` requires only 8 bytes. In conjunction with the direct packing (instead of pointer packing) in memory, the `ndarray` variable `np_lyst` requires much less memory than the Python variable `lyst`, and could engender much better cache coherence (read: average memory access time). Read more about it [here](https://webcourses.ucf.edu/courses/1249560/pages/python-lists-vs-numpy-arrays-what-is-the-difference). Additionally it is possible to write your own efficient C code implmentations that can be called from Python: read (much) more [here](https://medium.com/analytics-vidhya/beating-numpy-performance-by-extending-python-with-c-c9b644ee2ca8).

Computing the minimum of a collection is a straightforward linear complexity problem, $O(n)$. We'll try one more example, this time of sorting a `list/ndarray`, which you'll remember generally has complexity $O(n\log{}n)$.

In [ ]:
%%timeit
sorted(lyst)

In [ ]:
%%timeit
np.sort(np_lyst)

Both the [`sorted`](https://docs.python.org/3.7/howto/sorting.html#sortinghowto) and [`np.sort`](https://numpy.org/doc/stable/reference/generated/numpy.sort.html) methods are functions in the sense that they do not modify the argument collection (whether `list` or `ndarray`), but rather return a sorted copy of that collection. Alternative object-oriented in-place `sort()` and [`ndarray.sort()`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.sort.html#numpy.ndarray.sort) methods are also available. However, for the purposes of timing, sorting an already sorted list is pretty uninteresting.

In [ ]:
lyst.sort()
np_lyst.sort()

In [ ]:
%%timeit
sorted(lyst)

In [ ]:
%%timeit
np.sort(np_lyst)

<a id='essentials'></a>
## Numpy Essentials